# Showcase of the new LNHR DAC II QCoDeS driver (v0.2.0)

Copyright (c) Basel Precision Instruments GmbH (2025), written for the QCoDeS driver `Baspi_Lnhrdac2.py`, v0.2.0

...........................................................................................................................................................................................................................................

This notebook shows brief examples on the changes made and on how to use the new and improved QCoDeS driver for the Basel Precision Instruments LNHRDAC II.

## 1 - Imports and setting up a Station

In [1]:
from time import sleep

from qcodes.station import Station
from baspi_lnhrdac2 import BaspiLnhrdac2, BaspiLnhrdac2SWGConfig

The names of certain methods and variables are different to previous versions of the driver to fall in line with the QCoDeS recommendations. The behaviour of the DAC upon device creation has also changed.

In [2]:
station = Station()
DAC = BaspiLnhrdac2('LNHRDAC', 'TCPIP0::192.168.0.5::23::SOCKET')
station.add_component(DAC)


Connected to: Basel Precision Instruments GmbH (BASPI) LNHR DAC II (SP1060) - 24 channel version (serial:SN 10600000011, firmware:Revision 3.4.9u) in 6.48s
All channels have been turned off (1 MOhm Pull-Down to AGND) upon initialization and are pre-set to 0.0 V if turned on without setting a voltage beforehand.



'LNHRDAC'

In [3]:
print(DAC.awga.waveform_setpoints.get())
print(DAC.awga.waveform.get())

[]


ValueError: ("Shape of output is not consistent with setpoints. Output is shape (functools.partial(<bound method BaspiLnhrdac2Controller.get_awg_memory_size of <Baspi_Lnhrdac2_Controller.BaspiLnhrdac2Controller object at 0x000001A7BD390590>>, 'a'),) and setpoints are shape (functools.partial(<bound method BaspiLnhrdac2Controller.get_awg_memory_size of <Baspi_Lnhrdac2_Controller.BaspiLnhrdac2Controller object at 0x000001A7BD390590>>, 'a'),)", 'getting LNHRDAC_awga_waveform')

## 2 - Using the DAC as a DC Voltage source

When used as a DC Voltage source, the voltage and bandwidth of each channel are now controlled via QCoDeS parameters. Channels are also turned on and off via a parameter.

In [ ]:
DAC.ch14.voltage.set(5.86)
DAC.ch14.high_bandwidth.set("off")
DAC.ch14.enable.set("on")

voltage = DAC.ch14.voltage.get()
print(f"{voltage} V")

With all parameters of this driver, they can be controlled via different coding styles, as generally intended within QCoDeS.

In [ ]:
DAC.ch14.voltage(3.954)
DAC.ch14.high_bandwidth(False)
DAC.ch14.enable("ON")

DAC.all.enable("on)")

voltage = DAC.ch14.voltage()
print(f"{voltage} V")

## 3 - Using the DAC as an Arbitrary Waveform Generator (AWG)

The process of outputting an arbitrary waveform has two steps. On the first step the waveform has to be generated. Below it is shown how a waveform can be generated and saved to the device memory with the integrated Standard Waveform Generator (SWG) instrument module.

In [ ]:
config = BaspiLnhrdac2SWGConfig(
    shape = "sawtooth",
    frequency = 80,
    amplitude = 1.03,
    offset = 0.0,
    phase = 0.0
)

DAC.swg.configuration.set(config)
DAC.swg.apply("B")

After the waveform is saved to device memory, further control happens through the AWG instrument module. Through this module, device memory can be read or set. That also allows the user to not use the SWG but save a true arbitrary waveform to the DAC's memory.

In [ ]:
waveform = DAC.awga.waveform.get()
print(waveform)

The second step of outputting the waveform involves setting the parameters of the AWG module. The DAC has for AWG modules: AWG A, AWG B, AWG C and AWG D. In the code they are named in lower case and without spaces.

In [ ]:
# Make sure channel is actually enabled/ turned on
DAC.ch8.enable.set("on")

DAC.awgb.channel.set(8)
DAC.awgb.cycles.set(615)
DAC.awgb.trigger.set("disable")
DAC.awgb.enable.set("on")